In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timewarp_lib.load_model as lm
import timewarp_lib.train_utils as tu
import torch
import pandas as pd
import json
import numpy as np

In [ ]:
cmap = plt.get_cmap("viridis")

In [ ]:
datatype = "train"
dtype=torch.float
device="cpu"
batch_size=100
training_data_timing_noise=0

dataName = "../data/trainTest2DLetterARescaled.npz"
# the datafile has information on how the data was cleaned
# the learned/applied model is on clean data
# so we need to convert back to trajectory data
loaded_data_dict = np.load(dataName)

ydata = torch.tensor(loaded_data_dict[datatype],dtype=dtype).to(device)
np_ydata = ydata.detach().cpu().numpy()
num_trajs, numts, traj_channels = ydata.shape
tdata = torch.tensor(np.linspace(0,1,numts),dtype=dtype).to(device).expand(num_trajs,numts).unsqueeze(2)

torch_train_data = torch.utils.data.TensorDataset(tdata, ydata)
training_dataloader = torch.utils.data.DataLoader(torch_train_data, batch_size=batch_size, shuffle=True)

In [ ]:
ixs=(10,15)

In [ ]:
pose_mean = loaded_data_dict["pose_mean"]
pose_scaling = loaded_data_dict["pose_scaling"]

#saved_model_dir = clean_model_dirs[0]
#modelApplier = lm.LoadedModel(saved_model_dir)
#modeldatafileobj = np.load(f"{saved_model_dir}/saved_model_info.npz",allow_pickle=True)
#modeldata = {key : (modeldatafileobj[key] if key != "initialization_function" else True) for key in modeldatafileobj.keys()}
#num_timesteps = 200
#num_channels = 2
#latent_dim = (int)(modeldata["latent_dim"])
#
#hi=modelApplier.model

#embeds,_ = hi.encoder.encode(ydata[ixs,],tdata[ixs,])
#embeds = embeds.detach().cpu().numpy()
#
#scaled_ts = hi.scalar_timewarper.timewarp(ydata[ixs,],tdata[ixs,]).detach().cpu().numpy()

lambd = np.linspace(0,1,3)
#interpembeds = torch.tensor(np.array([embeds[0] * (1-l) + embeds[1] * (l) for l in lambd]))
#interpscaled_ts = torch.tensor(np.array([scaled_ts[0] * (1-l) + scaled_ts[1] * (l) for l in lambd]))
canonical_ts = torch.tensor(np.array([np.array(tdata[0,] * (1-l) + tdata[1,] * (l)) for l in lambd]))
#realtimed_trajs = hi.decoder.decode(interpembeds, interpscaled_ts).detach().numpy()
#canonicaltime_trajs = hi.decoder.decode(interpembeds, canonical_ts).detach().numpy()
naiveaverage_trajs = np.array([np_ydata[ixs[0]] * (1-l) + np_ydata[ixs[1]] * (l) for l in lambd])

lambd2 = np.linspace(0,1,11)
#full_interpembeds = torch.tensor(np.array([embeds[0] * (1-l) + embeds[1] * (l) for l in lambd2]))
full_canonical_ts = torch.tensor(np.array([np.array(tdata[0,] * (1-l) + tdata[1,] * (l)) for l in lambd2]))
#full_canonicaltime_trajs = hi.decoder.decode(full_interpembeds, full_canonical_ts).detach().numpy()

In [ ]:
from dtw import *
query = np_ydata[ixs[0],]
template = np_ydata[ixs[1]]
# symmetric2 is the normalizable, symmetric, with no local slope constraints. Since one diagonal step costs as much as the two equivalent steps along the sides, it can be normalized dividing by N+M (query+reference lengths). It is widely used and the default.
alignment = dtw(query,template, keep_internals=True)


In [ ]:
for i in range(len(alignment.index1)):
    start = query[alignment.index1[i]]
    end = template[alignment.index2[i]]
    color = "gray"
    plt.plot((start[0],end[0]),(start[1],end[1]),c=color,alpha=0.3)

trajs = []
plt.plot(query[:,0],query[:,1],c="k")
plt.plot(template[:,0]+0*4,template[:,1],c="k")
dtwaverage_trajs = []
all_interp_ts = []
unif_ts1 = np.linspace(0,1,len(query))
unif_ts2 = np.linspace(0,1,len(template))
for l in lambd:
    interp_weight = l
    interp_line = []
    interp_ts = []
    for j in range(len(alignment.index1)):
        start = query[alignment.index1[j]]
        end = template[alignment.index2[j]]
        start_t =unif_ts1[alignment.index1[j]]
        end_t =unif_ts2[alignment.index2[j]]
        interp_line.append((start*(1-interp_weight) + end*interp_weight))
        interp_ts.append((start_t*(1-interp_weight) + end_t*interp_weight))
    interp_line = np.array(interp_line)
    interp_ts = np.array(interp_ts)
    dtwaverage_trajs.append(interp_line)
    all_interp_ts.append(interp_ts)
dtwaverage_trajs = np.array(dtwaverage_trajs)
all_interp_ts = np.array(all_interp_ts)
plt.axis("equal")
plt.axis("off")

In [ ]:
arr = [tdata[0,] * (1-l) + tdata[1,] * (l) for l in lambd]
[print(a.shape) for a in arr]

In [ ]:
import matplotlib
font = {        'size'   : 22}

matplotlib.rc('font', **font)

In [ ]:
import matplotlib.lines as mlines 
#https://stackoverflow.com/questions/47391702/how-to-make-a-colored-markers-legend-from-scratch

In [ ]:
plot_order = [2,0,1]
for trajs,specified_ts,full_interp,name,ts  in [
    (dtwaverage_trajs,[210,228,238],dtwaverage_trajs,"dtw",all_interp_ts[1]),
    (naiveaverage_trajs,[118,130,139],naiveaverage_trajs,"naive",np.linspace(0,1,len(naiveaverage_trajs[0])))
    ]:

    linestyles = ["dotted", "dashed","dashdot"]
    
    f = plt.figure(figsize=(8,16))
    smallht = 0.37/2
    ax1 = f.add_axes([0.1, 0.6, 0.8, 0.4])  # add the left Axes
    ax2 = f.add_axes([0.1, 0.1, 0.8, smallht])  # add the bottomright Axes
    ax3 = f.add_axes([0.1, 0.5-smallht, 0.8, smallht])  # add the topright Axes

    alpha = 0.5
    threeixs = (0,1,-1)
    legend_lines = []
    shapes =["+",None,"x"]
    for i,traj in zip(plot_order,trajs[plot_order]):
        legend_lines.append(
            mlines.Line2D([], [], color=cmap(i/3), marker=shapes[i], label=f"---------",
                          markersize=10,markeredgecolor= cmap(i/3),markeredgewidth=1))

        ax1.plot(traj[:,0],traj[:,1],c=cmap(i/3),
                 linewidth=2 if i != 1 else 8,
                 alpha=alpha if i != 1 else 0.9, zorder=1,marker=shapes[i],markeredgecolor=cmap(i/3))
    for i,t in enumerate(specified_ts):
        t = (int)(t)
        ax1.plot(full_interp[:,t,0],full_interp[:,t,1],c="k",linestyle=linestyles[i])
        ax1.scatter(trajs[:,t,0],trajs[:,t,1],c=cmap((0,1/3,2/3)),
                    alpha=1,edgecolors= "k",linewidth=1, zorder=2)
    ax1.legend(handles=legend_lines,labelcolor="white", frameon=False)
    ax1.set_xlim(-2.5,3)
    ax1.set_ylim(-2,3.5)

    for pltax, axis, axisname in ((ax2,0,"X"),(ax3, 1,"Y")):
        for i,traj in zip(plot_order,trajs[plot_order]):
            pltax.plot(ts,traj[:,axis],c=cmap(i/3),
                     linewidth=2 if i != 1 else 5,
                     alpha=alpha if i != 1 else 0.9, zorder=1)
        for i,t in enumerate(specified_ts):
            pltax.plot(ts[t].repeat(3),trajs[:,t,axis],c="k",linestyle=linestyles[i])
            pltax.scatter(ts[t].repeat(3),trajs[:,t,axis],c=cmap((0,1/3,2/3)),
                        alpha=1,edgecolors= "k",linewidth=1, zorder=2)
        if axis==1:
            pltax.set_xticklabels([])
            pltax.set_ylim(-2,3.5)
        else:
            pltax.set_ylim(-2.5,3)
    plt.savefig(f"{name}InterpolationA.pdf",bbox_inches=matplotlib.transforms.Bbox([[0,0], [8,16]]))
    plt.show()